# MD5

In [8]:
import struct
import binascii
import math

class MD5:
    """
    MD5 Hash Algorithm Implementation
    """
    
    # Constants for MD5
    S = [
        7, 12, 17, 22, 7, 12, 17, 22, 7, 12, 17, 22, 7, 12, 17, 22,
        5,  9, 14, 20, 5,  9, 14, 20, 5,  9, 14, 20, 5,  9, 14, 20,
        4, 11, 16, 23, 4, 11, 16, 23, 4, 11, 16, 23, 4, 11, 16, 23,
        6, 10, 15, 21, 6, 10, 15, 21, 6, 10, 15, 21, 6, 10, 15, 21
    ]
    
    # Initialize variables
    A = 0x67452301
    B = 0xEFCDAB89
    C = 0x98BADCFE
    D = 0x10325476
    
    # Precomputed sine values (K table)
    K = [int(abs((2**32) * abs(math.sin(i + 1)))) & 0xFFFFFFFF for i in range(64)]
    
    @staticmethod
    def left_rotate(x, n):
        """Left rotate a 32-bit integer"""
        return ((x << n) | (x >> (32 - n))) & 0xFFFFFFFF
    
    @staticmethod
    def F(x, y, z):
        """First auxiliary function"""
        return (x & y) | (~x & z)
    
    @staticmethod
    def G(x, y, z):
        """Second auxiliary function"""
        return (x & z) | (y & ~z)
    
    @staticmethod
    def H(x, y, z):
        """Third auxiliary function"""
        return x ^ y ^ z
    
    @staticmethod
    def I(x, y, z):
        """Fourth auxiliary function"""
        return y ^ (x | ~z)
    
    @classmethod
    def pad_message(cls, message):
        """
        Pad the message according to MD5 specification
        """
        if isinstance(message, str):
            message = message.encode('utf-8')
        
        # Original length in bits
        original_length = len(message) * 8
        original_length_low = original_length & 0xFFFFFFFF
        original_length_high = (original_length >> 32) & 0xFFFFFFFF
        
        # Appending '1' bit
        message += b'\x80'
        
        # Appending '0' bits until length is 448 mod 512
        while (len(message) % 64) != 56:
            message += b'\x00'
        
        # Appending original length as 64-bit little-endian integer
        message += struct.pack('<II', original_length_low, original_length_high)
        
        return message
    
    @classmethod
    def process_block(cls, block, a, b, c, d):
        """
        Process a 512-bit block
        """
        # Breaking block into sixteen 32-bit words
        X = list(struct.unpack('<16I', block))
        
        # Saving original values
        AA, BB, CC, DD = a, b, c, d
        
        # Round 1
        for i in range(16):
            f = cls.F(b, c, d)
            g = i
            temp = d
            d = c
            c = b
            b = (b + cls.left_rotate((a + f + cls.K[i] + X[g]) & 0xFFFFFFFF, cls.S[i])) & 0xFFFFFFFF
            a = temp
        
        # Round 2
        for i in range(16, 32):
            f = cls.G(b, c, d)
            g = (5 * i + 1) % 16
            temp = d
            d = c
            c = b
            b = (b + cls.left_rotate((a + f + cls.K[i] + X[g]) & 0xFFFFFFFF, cls.S[i])) & 0xFFFFFFFF
            a = temp
        
        # Round 3
        for i in range(32, 48):
            f = cls.H(b, c, d)
            g = (3 * i + 5) % 16
            temp = d
            d = c
            c = b
            b = (b + cls.left_rotate((a + f + cls.K[i] + X[g]) & 0xFFFFFFFF, cls.S[i])) & 0xFFFFFFFF
            a = temp
        
        # Round 4
        for i in range(48, 64):
            f = cls.I(b, c, d)
            g = (7 * i) % 16
            temp = d
            d = c
            c = b
            b = (b + cls.left_rotate((a + f + cls.K[i] + X[g]) & 0xFFFFFFFF, cls.S[i])) & 0xFFFFFFFF
            a = temp
        
        # Adding this chunk's hash to result so far
        a = (a + AA) & 0xFFFFFFFF
        b = (b + BB) & 0xFFFFFFFF
        c = (c + CC) & 0xFFFFFFFF
        d = (d + DD) & 0xFFFFFFFF
        
        return a, b, c, d
    
    @classmethod
    def md5(cls, message):
        """
        Compute MD5 hash of the input message
        """
        # Initializing variables
        a = cls.A
        b = cls.B
        c = cls.C
        d = cls.D
        
        # Padding the message
        padded_message = cls.pad_message(message)
        
        # Processing the message in 512-bit chunks
        for i in range(0, len(padded_message), 64):
            block = padded_message[i:i+64]
            a, b, c, d = cls.process_block(block, a, b, c, d)
        
        # Producing the final hash value (little-endian)
        digest = struct.pack('<4I', a, b, c, d)
        
        return binascii.hexlify(digest).decode()


def md5_hash(message):
    """
    Convenience function to compute MD5 hash
    """
    return MD5.md5(message)


# Example usage and testing
if __name__ == "__main__":
    user_input = input("Enter a string: ")
    hash_result = md5_hash(user_input)
    print(f"MD5 hash: {hash_result}")

Enter a string:  Hello! My name is Pawan Pratap.


MD5 hash: 9f09f4cb18030daf84448133b30ecfda


# SHA-256

In [11]:
import struct
import binascii

class SHA256:
    """
    SHA-256 Hash Algorithm Implementation
    """
    
    # Initial hash values (first 32 bits of fractional parts of square roots of first 8 primes)
    H = [
        0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
        0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19
    ]
    
    # Constants (first 32 bits of fractional parts of cube roots of first 64 primes)
    K = [
        0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
        0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
        0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
        0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
        0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
        0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
        0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
        0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
    ]
    
    @staticmethod
    def right_rotate(n, b):
        """Right rotate a 32-bit integer n by b bits"""
        return ((n >> b) | (n << (32 - b))) & 0xffffffff
    
    @staticmethod
    def pad_message(message):
        """Pad the message according to SHA-256 specification"""
        if isinstance(message, str):
            message = message.encode('utf-8')
        
        # Original length in bits
        original_byte_len = len(message)
        original_bit_len = original_byte_len * 8
        
        # Appending '1' bit
        message += b'\x80'
        
        # Appending '0' bits until length is 448 mod 512
        while (len(message) * 8) % 512 != 448:
            message += b'\x00'
        
        # Appending original length as 64-bit big-endian integer
        message += struct.pack('>Q', original_bit_len)
        
        return message
    
    @classmethod
    def process_block(cls, block, h):
        """Process a 512-bit block"""
        # Preparing message schedule
        w = [0] * 64
        
        # Breaking chunk into sixteen 32-bit big-endian words
        for i in range(16):
            w[i] = struct.unpack('>I', block[i*4:(i+1)*4])[0]
        
        # Extending the sixteen 32-bit words into sixty-four 32-bit words
        for i in range(16, 64):
            s0 = cls.right_rotate(w[i-15], 7) ^ cls.right_rotate(w[i-15], 18) ^ (w[i-15] >> 3)
            s1 = cls.right_rotate(w[i-2], 17) ^ cls.right_rotate(w[i-2], 19) ^ (w[i-2] >> 10)
            w[i] = (w[i-16] + s0 + w[i-7] + s1) & 0xffffffff
        
        # Initializing working variables
        a, b, c, d, e, f, g, h_temp = h
        
        # Main loop
        for i in range(64):
            s1 = cls.right_rotate(e, 6) ^ cls.right_rotate(e, 11) ^ cls.right_rotate(e, 25)
            ch = (e & f) ^ (~e & g)
            temp1 = (h_temp + s1 + ch + cls.K[i] + w[i]) & 0xffffffff
            s0 = cls.right_rotate(a, 2) ^ cls.right_rotate(a, 13) ^ cls.right_rotate(a, 22)
            maj = (a & b) ^ (a & c) ^ (b & c)
            temp2 = (s0 + maj) & 0xffffffff
            
            h_temp = g
            g = f
            f = e
            e = (d + temp1) & 0xffffffff
            d = c
            c = b
            b = a
            a = (temp1 + temp2) & 0xffffffff
        
        # Adding compressed chunk to current hash value
        h[0] = (h[0] + a) & 0xffffffff
        h[1] = (h[1] + b) & 0xffffffff
        h[2] = (h[2] + c) & 0xffffffff
        h[3] = (h[3] + d) & 0xffffffff
        h[4] = (h[4] + e) & 0xffffffff
        h[5] = (h[5] + f) & 0xffffffff
        h[6] = (h[6] + g) & 0xffffffff
        h[7] = (h[7] + h_temp) & 0xffffffff
        
        return h
    
    @classmethod
    def sha256(cls, message):
        """Compute SHA-256 hash of the input message"""
        # Initializing hash values
        h = cls.H.copy()
        
        # Pad the message
        padded_message = cls.pad_message(message)
        
        # Processing the message in successive 512-bit chunks
        for i in range(0, len(padded_message), 64):
            block = padded_message[i:i+64]
            h = cls.process_block(block, h)
        
        # Producing the final hash value
        digest = b''.join(struct.pack('>I', x) for x in h[:8])
        
        return binascii.hexlify(digest).decode()


def sha256_hash(message):
    """Convenience function to compute SHA-256 hash"""
    return SHA256.sha256(message)


if __name__ == "__main__":
    user_input = input("Enter a string: ")
    hash_result = sha256_hash(user_input)
    print(f"SHA-256 hash: {hash_result}")

Enter a string:  My name is Tom Cruise.


SHA-256 hash: ffc8d9d55909518cb8f70871dd95d670efde6d57ac94bf5899df9a931dbec63e
